In [14]:
import sys
import yaml
import argparse
import pandas as pd
import numpy as np

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

import torch

from mcce import MCCE

dataset = OnlineCatalog("adult")

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.021740,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.000000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.000000,0.0,...,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
48827,0.301370,0.137428,0.800000,0.000000,0.0,...,0.0,1.0,1.0,0.0,1.0
48828,0.643836,0.209130,0.533333,0.000000,0.0,...,0.0,0.0,1.0,1.0,1.0
48829,0.287671,0.245379,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,1.0
48830,0.369863,0.048444,0.800000,0.054551,0.0,...,0.0,0.0,1.0,1.0,1.0


In [1]:
import sys
import yaml
import argparse
import pandas as pd
import numpy as np

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

import torch

from mcce import MCCE

dataset = ["adult"]
dataset = ['give_me_some_credit']
K = 1000
n_test = 100
seed = 1
results_all = None

# Use CARLA to load dataset and predictive model
print("Loading data from Carla...")

for data_name in dataset:
    dataset = OnlineCatalog(data_name)
    # y_col = dataset.target
    # x_col = dataset.raw.columns.to_list()
    # x_col.remove(y_col)
    
    # (1) Load predictive model and predict probabilities

    torch.manual_seed(0)
    ml_model = MLModelCatalog(
            dataset, 
            model_type="ann", 
            load_online=False, 
            backend="pytorch"
        )


    if data_name == 'adult':
        ml_model.train(
        learning_rate=0.002,
        epochs=20,
        batch_size=1024,
        hidden_size=[18, 9, 3],
        force_train=True, # don't forget to add this or it might load an older model from disk
        )
    elif data_name == 'give_me_some_credit':
        ml_model.train(
        learning_rate=0.002,
        epochs=20,
        batch_size=2048,
        hidden_size=[18, 9, 3],
        force_train=True, # don't forget to add this or it might load an older model from disk
        )
    elif data_name == 'compas':
        ml_model.train(
        learning_rate=0.002,
        epochs=25,
        batch_size=25,
        hidden_size=[18, 9, 3],
        force_train=True, # don't forget to add this or it might load an older model from disk
        )

    # (2) Find unhappy customers and choose which ones to make counterfactuals for
    
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:n_test]
    # test_factual_inverse = dataset.inverse_transform(test_factual)
    
    y_col = dataset.target
    features_and_response = dataset.df.columns
    cont_feat = dataset.continuous
    cat_feat = [x for x in features_and_response if x not in cont_feat] #  these have new names since encode_normalize_order_factuals()
    
    if data_name == 'adult': 
        fixed_features = ['age', 'sex_Male']
    elif data_name == 'give_me_some_credit':
        fixed_features = ['age']
    elif data_name == 'compas':
        fixed_features = ['age', 'sex_Male', 'race_Other']
    
    #  Create dtypes for MCCE()
    dtypes = dict([(x, "float") for x in cont_feat])
    for x in cat_feat:
        dtypes[x] = "category"
    df = (dataset.df).astype(dtypes)

    # (3) Fit MCCE object
    print("Fitting MCCE model...")
    mcce = MCCE(fixed_features=fixed_features, model=ml_model, seed=1)
    mcce.fit(df.drop(y_col, axis=1), dtypes)
    print("Generating counterfactuals with MCCE...")
    synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nr/samba/

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
Loading data from Carla...
balance on test set 0.9320676322926885, balance on test set 0.9323107818018143
Epoch 0/19
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.2694 Acc: 0.9321

test Loss: 0.2232 Acc: 0.9323

Epoch 1/19
----------
train Loss: 0.2133 Acc: 0.9321

test Loss: 0.2077 Acc: 0.9323

Epoch 2/19
----------
train Loss: 0.2024 Acc: 0.9321

test Loss: 0.1995 Acc: 0.9330

Epoch 3/19
----------
train Loss: 0.1978 Acc: 0.9337

test Loss: 0.1992 Acc: 0.9344

Epoch 4/19
----------
train Loss: 0.1968 Acc: 0.9349

test Loss: 0.2000 Acc: 0.9346

Epoch 5/19
----------
train Loss: 0.1960 Acc: 0.9350

test Loss: 0.2025 Acc: 0.9350

Epoch 6/19
----------
train Loss: 0.1954 Acc: 0.9346

test Loss: 0.1959 Acc: 0.9353

Epoch 7/19
----------
train Loss: 0.1950 Acc: 0.9349

test Loss: 0.1961 Acc: 0.9351

Epoch 8/19
----------
train Loss: 0.1946 Acc: 0.9351

test Loss: 0.1965 Acc: 0.9355

Epoch 9/19
----------
train Loss: 0.1945 Acc: 0.9352

test Loss: 0.1952 Acc: 0.9355

Epoch 10/19
----------
train Loss: 0.1945 Acc: 0.9349

test Loss: 0.1954 Acc: 0.9356

Epoch 11/19
----------
train Loss: 0.1944 Acc: 0.9349

test Loss: 0.1965 Acc: 0.9356



In [2]:
# (4) Postprocess generated counterfactuals
print("Postprocessing counterfactuals with MCCE...")
mcce.postprocess(df, synth_df, test_factual, y_col, scaler=dataset.inverse_transform, cutoff=0.5) # SOMETHING BREAKS HERE?

# 20 minutes for give_me_some_credit

Postprocessing counterfactuals with MCCE...


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [3]:
# mcce.results_sparse

In [4]:
# print(mcce.results_sparse.)
print(mcce.results_sparse.L0.mean())
print(mcce.results_sparse.L1.mean())
print(mcce.results_sparse.L2.mean())

4.69
0.5370523288089097
0.1746260798400699


In [5]:
results_sparse = mcce.results_sparse
results_sparse.index.rename('index', inplace=True)
results_sparse.groupby('index').size().sort_values(ascending=False)
#results_sparse.loc[64].iloc[0]#[[dataset.categorical + dataset.continuous]]

index
5295    1
2049    1
1792    1
1819    1
1828    1
       ..
3565    1
3608    1
3609    1
3752    1
9       1
Length: 100, dtype: int64